In [15]:
import os
from konlpy.tag import Mecab
import pandas as pd
import numpy as np
from collections import Counter
tokenizer = Mecab()

In [2]:
df = pd.read_csv('./data/train.csv')
print(df.head())


   idx      class                                       conversation
0    0      협박 대화  지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...
1    1      협박 대화  길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...
2    2  기타 괴롭힘 대화  너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...
3    3      갈취 대화  어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4    4      갈취 대화  저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...


In [22]:
# class별로 document 모으기
documents_by_class = df.groupby('class')['conversation'].apply(list)
cls_nm = documents_by_class.index.tolist()
cls_cnt = []
for class_name, documents in documents_by_class.items():
    result = []
    for content in documents :
        tokenlist = tokenizer.pos(content) 
        for word in tokenlist:
#                 if 'NNG' == word[1] or 'NNP' == word[1] or 'VV' == word[1]:#명사/동사
            result.append((word[0]+'__'+word[1].lower())) 
    cls_cnt.append(result)
assert len(cls_nm) == len(cls_cnt)

In [25]:
total_counts = Counter('')
temp = []
for i in range(len(cls_nm)):
    word_counts = Counter(cls_cnt[i])
    total_counts = total_counts + word_counts
    temp.append(word_counts)
temp2 =[]
for i in range(len(cls_nm)):
    word_ratios = {word: count / total_counts[word] if word in total_counts else 1 for word, count in temp[i].items() if count > 30}
    sorted_word_ratios = sorted(word_ratios.items(), key=lambda x: x[1], reverse=True)
    temp2.append(sorted_word_ratios)
for tt in range(len(temp2)) :
    print('\n\n'+cls_nm[tt], end=' : ')
    for i in range(min(30,len(temp2[tt]))) :
#         print(str(temp2[tt][i]), end=', ')
        print(str(temp2[tt][i][0]), end=', ')



갈취 대화 : 줄래__vv+ef, 차비__nng, 뒤져서__vv+ec, 빌려줘__vv+ec+vx+ec, 현금__nng, 용돈__nng, 시계__nng, 원__nnbc, 지갑__nng, 빌려줄__vv+ec+vx+etm, 내놔__vv+ec, 만__nr, 돌려줄__vv+etm, 빌려__vv+ec, 100__sn, 입금__nng, 억__nr, 돌려주__vv, 빌리__vv, 내놔__vv+ef, 주머니__nng, 계좌__nng, 핸드폰__nng, 가진__vv+etm, 벗__vv, 백__nr, 돈__nng, 내놓__vv, 선물__nng, 천__nr, 

기타 괴롭힘 대화 : 장애__nng, 못생긴__va+etm, 뚱뚱__xr, 인__xsn, 돼지__nng, 고객__nng, 닮__vv, 냄새__nng, 환불__nng, 놀리__vv, 씻__vv, 존나__mag, 따라__vv+ec, 찐따__nng, 괴롭히__vv, 배달__nng, 쟤__np, 병신__nng, 얘__np, 키__nng, 손님__nng, 거지__nng, 작__va, 아가씨__nng, 재밌__va, 개__nng, 공부__nng, 수업__nng, 음식__nng, 주제__nng, 

직장 내 괴롭힘 대화 : 연차__nng, 주임__nng, 사원__nng, 야근__nng, 부장__nng, 신입__nng, 휴가__nng, 인턴__nng, 입사__nng, 업무__nng, 프로젝트__nng, 상사__nng, 보고서__nng, 작성__nng, 대리__nng, 회식__nng, 회의__nng, 부서__nng, 과장__nng, 팀장__nng, 퇴근__nng, 박__nnp, 승진__nng, 주말__nng, 출근__nng, 팀__nng, 씨__nnb, 수정__nng, 점심__nng, 회사__nng, 

협박 대화 : 죽여__vv+ec, 장기__nng, 죽이__vv, 죽일__vv+etm, 고통__nng, 버릴__vx+etm, 헤어지__vv, 목숨__nng, 칼__nng, 살려__vv+ec, 협박__nng, 풀__vv, 건들__vv, 죽

# 나이브 베이지안

In [51]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score #정확도 계산

In [28]:
print(df.head())

   idx      class                                       conversation
0    0      협박 대화  지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...
1    1      협박 대화  길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...
2    2  기타 괴롭힘 대화  너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...
3    3      갈취 대화  어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4    4      갈취 대화  저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...


In [46]:
train = []
for index, row in df.iterrows():
#     print(row['conversation'])
    result = ''
    tokenlist = tokenizer.pos(row['conversation']) 
    for word in tokenlist:
#                 if 'NNG' == word[1] or 'NNP' == word[1] or 'VV' == word[1]:#명사/동사
        result = result + ' ' + word[0]+'__'+word[1].lower() 
    train.append(result)
print(len(train))
print(len(df))

3950
3950


In [38]:
encoder = LabelEncoder()
df['class'] = encoder.fit_transform(df['class'])

In [47]:
# 데이터셋 분리
X_train, X_val, y_train, y_val = train_test_split(train, df['class'], test_size=0.1, random_state=77)

In [48]:
print(len(X_train))
print(len(X_val))
print(len(y_train))
print(len(y_val))

3555
395
3555
395


In [49]:
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_val = vectorizer.transform(X_val) #테스트 데이터를 DTM으로 변환

In [52]:
model = MultinomialNB()
model.fit(X_train, y_train)
predicted = model.predict(X_val) #테스트 데이터에 대한 예측
print("NB 정확도:", accuracy_score(y_val, predicted)) #예측값과 실제값 비교

NB 정확도: 0.8683544303797468
